# Ngày 6: SPARK SQL - DATAFRAME

### 1. Giới thiệu về Spark SQL

1.1. Định Nghĩa

- Spark SQL là module xử lý dữ liệu có cấu trúc trong Apache Spark
- Cung cấp giao diện để làm việc với dữ liệu có cấu trúc và bán cấu trúc

1.2 Ưu Điểm

- Tích hợp truy vấn SQL với xử lý dữ liệu phân tán
- Hiệu suất cao nhờ Catalyst Optimizer
- Hỗ trợ nhiều nguồn dữ liệu

### 2. Kiến Trúc Spark SQL

2.1 Thành Phần Chính

- DataFrame API
- Dataset API
- SQL Interface

2.2 Catalyst Optimizer\
Quá trình tối ưu hóa truy vấn
Các giai đoạn:
- Phân tích cú pháp
- Kiểm tra logic
- Tối ưu hóa kế hoạch
- Sinh mã thực thi

2.3 Tungsten Execution Engine

- Tối ưu hóa bộ nhớ và xử lý
- Quản lý bộ nhớ hiệu quả
- Giảm overhead GC

### 3. DataFrame và Dataset

3.1 So Sánh
- RDD: Không có cấu trúc, hiệu suất thấp
- DataFrame: Có cấu trúc, tối ưu hơn
- Dataset: Kết hợp ưu điểm RDD và DataFrame
  
3.2 Các Phép Biến Đổi
- Transformation: select, filter, groupBy
- Action: show, count, collect

### 4. Tạo DataFrame

4.1 Từ RDD
```python
# Chuyển RDD sang DataFrame
rdd = sc.parallelize([(1, "Alice"), (2, "Bob")])
df = spark.createDataFrame(rdd, ["id", "name"])
```

4.2 Từ File

```python
# CSV
df_csv = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("data.csv")

# JSON
df_json = spark.read.json("data.json")
```

4.3 Từ JDBC

```python
df_jdbc = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost/db") \
    .option("dbtable", "users") \
    .option("user", "username") \
    .option("password", "password") \
    .load()
```

### 5. Truy vấn SQL

5.1 Đăng kí view

**Tạo view tạm thời**
df.createOrReplaceTempView("social_posts")

**Truy vấn SQL**
```python
result = spark.sql("""
    SELECT 
        date, 
        COUNT(*) as post_count,
        AVG(likes) as avg_likes
    FROM social_posts
    GROUP BY date
""")
```

5.2 Các hàm thông dụng

- Aggregate: sum(), avg(), count()
- Window Functions
- Joins
- Subqueries

## Bài Tập Thực Hành

Bài 1: Tạo DataFrame từ một file CSV chứa dữ liệu mạng xã hội và thực hiện các truy vấn SQL đơn giản (ví dụ: đếm số bài đăng theo ngày, tìm các hashtag phổ biến).

In [4]:
# Khởi tạo Spark Session
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

# Tạo Spark Session
spark = SparkSession.builder \
    .appName("Social Media Analysis") \
    .getOrCreate()

# Đọc dữ liệu
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("material_2/social_media.csv")

# Hiển thị cấu trúc DataFrame
df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- username: string (nullable = true)
 |-- post_date: date (nullable = true)
 |-- content: string (nullable = true)
 |-- likes: integer (nullable = true)
 |-- shares: integer (nullable = true)
 |-- hashtags: string (nullable = true)



In [6]:
# Phân tích các hagstags phổ biến
df.select(explode(split("hashtags", ",\s*")).alias("hashtag")) \
    .groupBy("hashtag") \
    .count() \
    .orderBy(desc("count")) \
    .show()

+--------------------+-----+
|             hashtag|count|
+--------------------+-----+
|#startup #technology|    1|
|      #happy #travel|    1|
|#datascience #coding|    1|
|     #bigdata #spark|    1|
|       #weekend #fun|    1|
+--------------------+-----+



In [8]:
# Phân tích các người dùng phổ biến
# Top người dùng theo engagement
user_engagement = df.groupBy("username") \
    .agg(
        count("*").alias("total_posts"),
        sum("likes").alias("total_likes"),
        sum("shares").alias("total_shares")
    ) \
    .orderBy(col("total_likes").desc())

user_engagement.show(5)

+----------+-----------+-----------+------------+
|  username|total_posts|total_likes|total_shares|
+----------+-----------+-----------+------------+
|  john_doe|          2|        450|          55|
|  emma_dev|          1|        400|          60|
| alex_tech|          1|        300|          45|
|jane_smith|          1|        150|          15|
+----------+-----------+-----------+------------+



In [9]:
# Thống kê hoạt động theo ngày
daily_activity = df.groupBy("post_date") \
    .agg(
        count("*").alias("total_posts"),
        avg("likes").alias("avg_likes"),
        avg("shares").alias("avg_shares")
    ) \
    .orderBy("post_date")

daily_activity.show()

+----------+-----------+---------+----------+
| post_date|total_posts|avg_likes|avg_shares|
+----------+-----------+---------+----------+
|2024-01-15|          2|    200.0|      22.5|
|2024-01-16|          2|    250.0|      35.0|
|2024-01-17|          1|    400.0|      60.0|
+----------+-----------+---------+----------+



In [10]:
# Truy vấn sql
# Đăng ký view để truy vấn SQL
df.createOrReplaceTempView("social_posts")

# Truy vấn SQL phức tạp
advanced_query = spark.sql("""
    SELECT 
        username,
        COUNT(*) as post_count,
        SUM(likes) as total_likes,
        RANK() OVER (ORDER BY SUM(likes) DESC) as engagement_rank
    FROM social_posts
    GROUP BY username
    HAVING post_count > 1
""")

advanced_query.show()

24/11/27 00:05:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/27 00:05:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/27 00:05:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/27 00:05:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/27 00:05:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/27 00:05:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------+----------+-----------+---------------+
|username|post_count|total_likes|engagement_rank|
+--------+----------+-----------+---------------+
|john_doe|         2|        450|              1|
+--------+----------+-----------+---------------+



In [11]:
spark.stop()